In [ ]:
#以下はgoogle colab内で実行しない場合は必要ないが、作業フォルダ内には移動しておく
"""
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# 作業フォルダに移動
# %cd "/content/drive/MyDrive/Colab Notebooks/~~~~
"""

'\n# Google Driveをマウント\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n\n# 作業フォルダに移動\n# %cd "/content/drive/MyDrive/Colab Notebooks/~~~~\n'

In [6]:
!pip install selenium -q
!apt-get update -q
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y -q
!rm google-chrome-stable_current_amd64.deb

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://dl.google.com/linux/chrome/deb stable InRelease
Fetched 3,632 B in 5s (697 B/s)
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [ ]:

# シラバス検索のどの段階かを管理するためのステップ定義
class SyllabusStep(Enum):
    WAIT_YEAR = auto()      # 年度待ち
    WAIT_GAKUBU = auto()    # 学部待ち
    WAIT_GAKKI = auto()     # 学期待ち
    WAIT_GAKUNEN = auto()   # 学年待ち
    WAIT_KEYWORD = auto()   # キーワード待ち

# Botの現在の状態を保持するグローバル変数
state = State.NORMAL
syllabus_step = None
syllabus_params = {} # ユーザーからの回答を保存する辞書


In [ ]:

import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import requests
from bs4 import BeautifulSoup
from google.colab import userdata  # Colab内だけで安全にID/PASSを取得

# 大学サイト・バス時刻表は仮URL
UNIVERSITY_URL = "https://example.com/campusweb"
BUS_SCHEDULE_URL = "https://example.com/bus_schedule.html"

# WebDriverをセットアップ
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)
    print("ドライバー起動完了")
    return driver, wait

# ログインしてシラバス参照ページに移動
def login_and_sizi(driver, wait):
    print("シラバスサイトにログイン中")
    driver.get(UNIVERSITY_URL)
    #googleのシークレット機能を使ってコード内に直接passwordを打たない形にする
    USER_ID = userdata.get("USER_ID")
    PASSWORD = userdata.get("PASSWORD")
    wait.until(EC.presence_of_element_located((By.NAME, "userName"))).send_keys(USER_ID)
    wait.until(EC.presence_of_element_located((By.NAME, "password"))).send_keys(PASSWORD)
    wait.until(EC.element_to_be_clickable((By.NAME, "image"))).click()
    print("ログイン成功")
    # フレーム切替やシラバス参照リンクのクリックは省略・例として残す
    print("シラバスページへ遷移完了（フレーム操作は省略）")


"""
ここまで実行して「ログイン成功」と表示されれば、
自分の画面上では確認できなくても、裏で自動操作は正常に行われている
"""



"""
########################################################################################################################################
# バス時刻表取得（仮のHTMLで対応）
def get_bus_schedules():
    try:
        response = requests.get(BUS_SCHEDULE_URL)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        schedules = {"~~~~": [], "~~~~~": []} #~~~はhtml内の駅名
        print("バス時刻表の取得完了（仮）")
        return schedules
    except Exception as e:
        print(f"バス時刻表の取得に失敗: {e}")
        return None

# 授業時限から間に合うバスを計算
def maniau_bus(yobi_jigen_str, bus_schedules):
    if not bus_schedules:
        return "バス時刻を取得できなかった"
    class_kaisi_jikan = {"1": 8*60+50, "2": 10*60+45, "3": 13*60+15, "4": 15*60+10, "5": 17*60+5}
    match = re.search(r'\d', yobi_jigen_str)
    if not match: return "時限情報が不明"
    jigen = match.group(0)
    kaisi_jikan = class_kaisi_jikan.get(jigen)
    if not kaisi_jikan: return f"{jigen}限の開始時刻が不明"
    osusume = []
    stations = {
        "~~~駅": {"schedule": bus_schedules.get("hachioji", []), "allowance": 20},
        "~~~駅": {"schedule": bus_schedules.get("minamino", []), "allowance": 10}
    }
    for station_name, info in stations.items():
        bus_time_str = "間に合うバスなし"
        limit_time = kaisi_jikan - info["allowance"]
        for osoi_jun in reversed(info["schedule"]):
            if osoi_jun <= limit_time:
                syuppatu_hour = osoi_jun // 60
                syuppatu_min = osoi_jun % 60
                bus_time_str = f"{syuppatu_hour:02d}:{syuppatu_min:02d}発"
                break
        osusume.append(f"{station_name}: {bus_time_str}")
    return ", ".join(osusume)

"""

# シラバス検索（簡略化）
def kensaku_jikkou(driver, wait, year, gakki, gakubu, gakunen, subject_keyword="", get_details=False, bus_schedules=None):
    result_lines = []
    nendo_input = driver.find_element(By.ID, "nendo")
    nendo_input.clear()
    nendo_input.send_keys(str(year))
    Select(driver.find_element(By.ID, "jikanwariShozokuCode")).select_by_value(str(gakubu))
    Select(driver.find_element(By.ID, "gakkiKubunCode")).select_by_value(str(gakki))
    Select(driver.find_element(By.ID, "nenji")).select_by_value(str(gakunen))
    driver.find_element(By.ID, "kaikoKamokunm").clear()
    driver.find_element(By.XPATH, "//input[@value=' 検索開始 ']").click()
    print("検索結果を表示中...")
    try:
        wait.until(EC.visibility_of_element_located((By.XPATH, "//table[@class='normal']")))
        print("検索結果の表示を確認")
    except TimeoutException:
        print("--- 検索結果: 0件 ---")
        return "検索結果: 0件"
    retu_xpath = "//table[@class='normal']//tr"
    all_retu = driver.find_elements(By.XPATH, retu_xpath)
    result_lines.append(f"--- {len(all_retu) - 1}件の検索ヒット。 ---\n")
    syorisuru_gyou = []
    if subject_keyword:
        result_lines.append(f"--- ここから科目名に「{subject_keyword}」を含むものを探します ---\n")
        filtered_retu = []
        for row in all_retu[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 7 and subject_keyword in cells[5].text:
                filtered_retu.append(row)
        if not filtered_retu:
            result_lines.append("条件に一致する科目は見つかりませんでした。")
            return "".join(result_lines)
        syorisuru_gyou = filtered_retu
        result_lines.append(f"--- 絞り込み結果: {len(syorisuru_gyou)}件 ---\n")
    else:
        syorisuru_gyou = all_retu[1:]
    if not get_details:
        for row in syorisuru_gyou:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 7:
                result_lines.append(f"科目名: {cells[5].text}, 担当: {cells[6].text}\n")
        return "".join(result_lines)
    result_lines.append(f"--- {len(syorisuru_gyou)}件の科目の詳細情報を処理する ---\n")
    target_indices = [all_retu.index(row) for row in syorisuru_gyou]
    for i in target_indices:
        all_retu_reloaded = driver.find_elements(By.XPATH, retu_xpath)
        ima_retu = all_retu_reloaded[i]
        cells = ima_retu.find_elements(By.TAG_NAME, "td")
        yobi_jigen_text = cells[3].text.replace("\n", "")
        kamoku_mei = cells[5].text
        kyoin_mei = cells[6].text
        result_lines.append(f"\n{i+1}件目: 「{kamoku_mei}」の詳細情報を取得中\n")
        ima_retu.find_element(By.XPATH, ".//input[@value='参照']").click()
        wait.until(EC.visibility_of_element_located((By.XPATH, "//th[text()='教室']")))
        kyoshitsu_info = driver.find_element(By.XPATH, "//th[text()='教室']/following-sibling::td").text
        mani = maniau_bus(yobi_jigen_text, bus_schedules)
        match = re.search(r'\d', yobi_jigen_text)
        jigen_display = f"{match.group(0)}限" if match else "不明"
        result_lines.append(f"  - 科目名: {kamoku_mei}\n")
        result_lines.append(f"  - 担当: {kyoin_mei}\n")
        result_lines.append(f"  - 教室: {kyoshitsu_info}\n")
        result_lines.append(f"  - 時限: {jigen_display}\n")
        result_lines.append(f"  - 遅刻回避可能なバス: {mani}\n")
        driver.back()
        print("  - フレームを再設定中")
        driver.switch_to.default_content()
        wait.until(EC.frame_to_be_available_and_switch_to_it(2))
        wait.until(EC.visibility_of_element_located((By.XPATH, "//table[@class='normal']")))
        print("  - 一覧ページに戻る")
    return "".join(result_lines)

def syllabus_sukureipingu(params):
    driver = None
    try:
        bus_schedules = get_bus_schedules()
        driver, wait = setup_driver()
        login_and_sizi(driver, wait)
        year, gakubu, gakki, gakunen, keyword = params
        get_details = True if keyword else False
        result_string = kensaku_jikkou(driver, wait, year, gakki, gakubu, gakunen,
                                      subject_keyword=keyword, get_details=get_details, bus_schedules=bus_schedules)
        return result_string
    #例外処理
    except Exception as e:
        error_message = f"処理中にエラーが発生しました: {e}"
        print(error_message)
        if driver:
            try:
                driver.save_screenshot('error_screenshot.png')
                print("'error_screenshot.png' にスクリーンショットを保存")
            except:
                pass
        return error_message
    finally:
        if driver:
            print("\nドライバーを終了")
            driver.quit()


SyntaxError: incomplete input (716964754.py, line 47)

In [ ]:
import asyncio #非同期処理（ある処理の完了を待たずに次の処理を実行できる方式）が可能

async def generateResponse(message):
    global state, syllabus_step, syllabus_params
    inputText = message.content

    # シラバス検索モードの処理
    #入力後基本的にその下に記述しているコードを実行
    if state == State.SYLLABUS:
        if syllabus_step == SyllabusStep.WAIT_YEAR:
            syllabus_params['year'] = inputText
            syllabus_step = SyllabusStep.WAIT_GAKUBU
            await message.channel.send("次に学部コードを入力してください")
            return
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        elif syllabus_step == SyllabusStep.WAIT_GAKUBU:
            syllabus_params['gakubu'] = inputText
            syllabus_step = SyllabusStep.WAIT_GAKKI
            await message.channel.send("次に学期を入力してください（例: 前期, 後期）")
            return
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        elif syllabus_step == SyllabusStep.WAIT_GAKKI:
            syllabus_params['gakki'] = inputText
            syllabus_step = SyllabusStep.WAIT_GAKUNEN
            await message.channel.send("次に対象学年を数字で入力してください（例: 1, 2, 3, 4）")
            return
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        elif syllabus_step == SyllabusStep.WAIT_GAKUNEN:
            syllabus_params['gakunen'] = inputText
            syllabus_step = SyllabusStep.WAIT_KEYWORD
            await message.channel.send("最後に検索したい科目名のキーワードを入力してください。\n（特に指定がなければ「なし」と入力）")
            return
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        elif syllabus_step == SyllabusStep.WAIT_KEYWORD:
            if inputText in ["なし", "ない", "無し", "いらない"]:
                syllabus_params['keyword'] = ""
            else:
                syllabus_params['keyword'] = inputText

            await message.channel.send(f"ありがとうございます。以下の条件で検索します。\n```{syllabus_params}```\nウェブサイトにアクセスするため、30秒～1分ほどかかります...")

            # ユーザー入力をウェブサイトで使われる内部コードに変換するための対応表
            #以下は検索、出力
            gakubu_map = {
                "~~学部": "~~", "~~": "~~",
                "~~学部": "~~", "~~": "~~",
                "~~学部": "~~", "~~": "~~",
                "~~学部": "~~", "~~": "~~",
                "~~学部": "~~", "~~": "~~",
                "~~学部": "~~", "~~": "~~",
                "~~学部": "~~", "~~": "~~"
            }
            gakki_map = {"前期": "1", "後期": "2"}
            nenji_map = {"1年": "1", "1": "1", "2年": "2", "2": "2", "3年": "3", "3": "3", "4年": "4", "4": "4"}

            # 変換を実行
            gakubu_code = gakubu_map.get(syllabus_params.get("gakubu").upper(), syllabus_params.get("gakubu"))
            gakki_code = gakki_map.get(syllabus_params.get("gakki"), syllabus_params.get("gakki"))
            gakunen_code = nenji_map.get(syllabus_params.get("gakunen"), syllabus_params.get("gakunen"))

            loop = asyncio.get_event_loop() # 現在のイベントループ
            params_tuple = (
                syllabus_params["year"], gakubu_code, gakki_code,
                gakunen_code, syllabus_params["keyword"]
            )
            # 同期関数であるウェブスクレイピング処理を別スレッドで実行し、完了待ち。
            result_text = await loop.run_in_executor(None, syllabus_sukureipingu, params_tuple)

            if len(result_text) > 1950:
                    await message.channel.send("結果が長すぎるため、分割して送信します。")
                    for i in range(0, len(result_text), 1950):
                        await message.channel.send(f"```{result_text[i:i+1950]}```")
            else:
                    await message.channel.send(f"```{result_text}```")

            state = State.NORMAL
            syllabus_step = None
            syllabus_params = {}
            return

    # --- 通常モードでのコマンド受付 ---
    #ここでシラバスモードに入る。年度を送信すると下のセルにあるSyllabusStep.WAIT_YEARを実行する
    if "シラバス" in inputText:
        state = State.SYLLABUS
        syllabus_step = SyllabusStep.WAIT_YEAR
        syllabus_params = {}
        await message.channel.send("シラバス検索を開始します。年度を数字で入力してください（例: 2025）")
        return


In [13]:
!pip install discord.py
!pip install nest_asyncio

In [14]:
# @title ステップ5-1：Discord Botのクライアント準備
import discord
import nest_asyncio
import settings

nest_asyncio.apply()
intents = discord.Intents.default()
intents.message_content = True
client = discord.Client(intents=intents)


In [ ]:
#@title DiscordBotがメッセージを受信した時の処理

############################# 消さない ################################################################################################
@client.event
async def on_message(message):  # メッセージを受信したときのイベント
    if message.author.bot:  # 無限ループしないようにメッセージを送ったのがBot自身なら処理をスキップ
        return
    if message.channel.name != settings.channel_name: #####################消すな#################################
        return

    if message.content.strip():
      try:
            output = await generateResponse(message)
            if output is not None:
                await message.channel.send(output)
      except Exception as e:
            print(f"エラーが発生しました: {e}")# メッセージが空でない場合
            # エラーの詳細をコンソールに表示するため、トレースバックも出力
            import traceback
            traceback.print_exc()
            await message.channel.send("申し訳ありません、処理中にエラーが発生しました。")
######################################################################################################################################


NameError: name 'client' is not defined

### Botの起動

In [16]:
@client.event
async def on_ready():  # Botが準備完了したときのイベント
    print('ログインしました')

await client.start(settings.token,reconnect=True)  # Botを起動

ログインしました


CancelledError: 